In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
# Carregar os dados
df = pd.read_csv("EDA_Manipulados_201401.csv")

In [ ]:
# Converter ano e mês para um índice de tempo
df['DATA'] = pd.to_datetime(df['ANO_VI'].astype(str) + '-' + df['MES_VE'].astype(str))
df.set_index('DATA', inplace=True)
# Selecionar a variável alvo (exemplo: 'QTD_AT')
df = df[['QTD_AT']]
df.dropna(inplace=True)

In [ ]:
# Normalizar os dados
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

In [ ]:
# Criar sequência de dados para LSTM
def create_sequences(data, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        targets.append(data[i+seq_length])
    return np.array(sequences), np.array(targets)

seq_length = 3  # Usamos os últimos 3 meses para prever o próximo
X, y = create_sequences(df_scaled, seq_length)

In [ ]:
# Dividir em treino e teste
test_size = int(len(X) * 0.2)
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]

In [ ]:
# Construção do modelo LSTM
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Fazer previsões
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
# Avaliar a performance
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Real')
plt.plot(y_pred, label='Previsto')
plt.legend()
plt.title('Previsão de Séries Temporais com LSTM')
plt.show()